In [1]:
import re

def ints_in_line(s):
    return list(map(int, re.findall(r"-?\d+", s)))

In [2]:
filename = "input.txt"
# filename = "example1.txt"

with open(filename, "r") as f:
    input_ = f.readlines()

from collections import namedtuple

Position = namedtuple('Position', ['x', 'y', 'z'])
Velocity = namedtuple('Velocity', ['x', 'y', 'z'])
Trajectory = namedtuple('Trajectory', ['p', 'v'])

numbers = [ints_in_line(l) for l in input_]
vals = [Trajectory(Position(*v[:3]), Velocity(*v[3:])) for v in numbers]
print(vals[:5])

[Trajectory(p=Position(x=368925240582247, y=337542061908847, z=298737178993847), v=Velocity(x=21, y=-126, z=-9)), Trajectory(p=Position(x=287668477092999, y=306868689869154, z=240173335647821), v=Velocity(x=-21, y=-15, z=29)), Trajectory(p=Position(x=172063062341522, y=378381220662744, z=223621999511007), v=Velocity(x=-25, y=-38, z=-64)), Trajectory(p=Position(x=173207142739382, y=380138705823962, z=212955454913987), v=Velocity(x=161, y=-145, z=90)), Trajectory(p=Position(x=247022614384315, y=185370784055125, z=147179140524802), v=Velocity(x=-69, y=522, z=316))]


# Part 1

In [3]:
from itertools import combinations

In [4]:

import numpy as np

def slope(trajectory):
    if trajectory.v.x != 0:
        return trajectory.v.y / trajectory.v.x
    else:
        return np.sign(trajectory.v.y) * np.inf

def get_time(trajectory, p):
    if trajectory.v.x != 0:
        return (p.x - trajectory.p.x) / trajectory.v.x
    else:
        return (p.y - trajectory.p.y) / trajectory.v.y


def intersection(traj1, traj2):
    m1 = slope(traj1)
    m2 = slope(traj2)
    if m1 > m2:
        # let traj1 have the smaller slope
        traj1, traj2 = traj2, traj1
        m1 = slope(traj1)
        m2 = slope(traj2)
    x1, y1 = traj1.p.x, traj1.p.y
    x2, y2 = traj2.p.x, traj2.p.y
    # if m1 == m2 or m1 == -m2:
    #     return None
    if m2 == np.inf:
        return Position(x2, m1 * (x2-x1) + y1, None)
    if m1 == m2 or m1 == -m2:
        x = (x1 + x2) / 2
    else:
        x = ((y2 - m2*x2) - (y1 - m1*x1)) / (m1-m2)
    return Position(x, m1 * (x-x1) + y1, None)



In [5]:
LBOUND = 200000000000000
UBOUND = 400000000000000

def in_bounds(pos):
    return LBOUND <= pos.x <= UBOUND and LBOUND <= pos.y <= UBOUND

In [6]:
count = 0
for t1,t2 in combinations(vals, 2):
    i = intersection(t1, t2)
    if i is None or not in_bounds(i):
        continue
    time1 = get_time(t1, i)
    time2 = get_time(t2, i)
    if time1 > 0 and time2 > 0:
        count += 1
print(count)




15593


# Part 2

In [7]:
import sympy as sp
from sympy import symbols

num_t = 3  # 3 eqs gives a 9x9 system
t = [symbols(f't{i}') for i in range(num_t)]
px, py, pz, vx, vy, vz = symbols('px py pz vx vy vz')

def soe_sym():
    eqs = []
    for i, traj in enumerate(vals[:num_t]):
        # p + t * v = pi + t * vi
        eqs.extend([
            px + t[i]*(vx - traj.v.x) - traj.p.x,
            py + t[i]*(vy - traj.v.y) - traj.p.y,
            pz + t[i]*(vz - traj.v.z) - traj.p.z,
        ])
    return eqs


In [8]:
eqs_sym = soe_sym()
solution = sp.solve(eqs_sym)
solution

[{px: 155272940103072,
  py: 386989974246822,
  pz: 214769025967097,
  t0: 932979478075,
  t1: 488544416937,
  t2: 61054989958,
  vx: 250,
  vy: -179,
  vz: 81}]

In [9]:
s = solution[0]
print(s[px] + s[py] + s[pz])

757031940316991


In [10]:
for x in [s[px], s[py], s[pz]]:
    print(x, LBOUND <= x <= UBOUND)

155272940103072 False
386989974246822 True
214769025967097 True


In [11]:
X = sp.Matrix(eqs_sym)
Y = sp.Matrix([px,py,pz,vx,vy,vz,*t[:3]])
J = X.jacobian(Y)

In [12]:
J

Matrix([
[1, 0, 0, t0,  0,  0,  vx - 21,       0,       0],
[0, 1, 0,  0, t0,  0, vy + 126,       0,       0],
[0, 0, 1,  0,  0, t0,   vz + 9,       0,       0],
[1, 0, 0, t1,  0,  0,        0, vx + 21,       0],
[0, 1, 0,  0, t1,  0,        0, vy + 15,       0],
[0, 0, 1,  0,  0, t1,        0, vz - 29,       0],
[1, 0, 0, t2,  0,  0,        0,       0, vx + 25],
[0, 1, 0,  0, t2,  0,        0,       0, vy + 38],
[0, 0, 1,  0,  0, t2,        0,       0, vz + 64]])